## Import Vision Libraries

In [1]:
#First run only
#!pip install google-cloud-vision
import os
import io
#Cred key
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '../1_Vision/Keys/glossy-radio-377415-d797ca03c624.json'

from google.cloud import vision
from PIL import Image, ImageDraw

In [5]:
image_path = os.path.abspath('./image02.jpg')

### Get crop vertices

In [2]:
def get_crop_hint(path):
    client = vision.ImageAnnotatorClient()
    with io.open(path, 'rb') as image_file:
        content = image_file.read()
    image = vision.Image(content=content)
    crop_hints_params = vision.CropHintsParams(aspect_ratios=[1.1])
    image_context = vision.ImageContext(crop_hints_params=crop_hints_params)
    response = client.crop_hints(image=image, image_context=image_context)
    hints = response.crop_hints_annotation.crop_hints
    vertices = hints[0].bounding_poly.vertices
    return vertices

### Draw bounding poly

In [3]:
def draw_hint(image_file):
    vects = get_crop_hint(image_file)
    im = Image.open(image_file)
    draw = ImageDraw.Draw(im)
    draw.polygon([
        vects[0].x, vects[0].y,
        vects[1].x, vects[1].y,
        vects[2].x, vects[2].y,
        vects[3].x, vects[3].y], None, 'red')
    im.save('output-hint.jpg', 'JPEG')
    print('Saved new image to output-hint.jpg')

### Crop and save as new

In [7]:
def crop_to_hint(image_file):
    vects = get_crop_hint(image_file)
    im = Image.open(image_file)
    im2 = im.crop([vects[0].x, vects[0].y,
                  vects[2].x - 1, vects[2].y - 1])
    im2.save('output-crop2.jpg', 'JPEG')
    print('Saved new image to output-crop.jpg')

In [8]:
crop_to_hint(image_file=image_path)

Saved new image to output-crop.jpg


In [11]:
orig_image = Image.open(image_path)
vects = get_crop_hint(image_path)

In [13]:
print("x = {}, y = {}, x2 = {}, y2 = {}".format(vects[0].x,vects[0].y,vects[2].x,vects[2].y))
#im_crop = im.crop((left, upper, right, lower))
x = vects[0].x
y = vects[0].y
x2 = vects[2].x
y2 = vects[2].y
print("x = {}, y = {}, x2 = {}, y2 = {}".format(x,y,x2,y2))

x = 0, y = 129, x2 = 783, y2 = 574
x = 0, y = 129, x2 = 783, y2 = 574


In [17]:
orig_image = Image.open(image_path)
obj_image = orig_image.crop((x, y, x2, y2))
obj_image.save("crop-1.png")
width = x2 - x
height = y2 - y
white_space = Image.new("RGB", (width, height), (255, 255, 255))
orig_image.paste(white_space, (x, y))
orig_image.save("new_orig.png")